In [2]:
# FINAL TEST SET EVALUATION (test_public_80)
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
import glob
import pandas as pd
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm
import time

# --- CONFIGURATION ---
TEST_CONFIG = {
    "TEST_DIR": r"c:/Users/Rayan/Downloads/Compressed/unzipped_files-20260214T145034Z-1-001/unzipped_files/test_public_80",
    "MODEL_PATH": r"c:/Users/Rayan/Downloads/Compressed/unzipped_files-20260214T145034Z-1-001/unzipped_files/submission1/best_model_unetpp.pth",
    "OUTPUT_DIR": "./test_results_public_80",
    "IMG_SIZE": 512,
    "ENCODER": "efficientnet-b4",
    "NUM_CLASSES": 10,
    "DEVICE": torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

# Visualization Colors (BGR)
COLOR_MAP = {
    0: (0, 0, 0),       1: (34, 139, 34),   2: (0, 255, 0),     3: (140, 180, 210),
    4: (43, 90, 139),   5: (0, 128, 128),   6: (19, 69, 139),   7: (128, 128, 128),
    8: (45, 82, 160),   9: (235, 206, 135)
}
CLASS_NAMES = ['Background', 'Trees', 'Lush Bushes', 'Dry Grass', 'Dry Bushes',
               'Ground Clutter', 'Logs', 'Rocks', 'Landscape', 'Sky']

ID_TO_INDEX = {
    100: 0, 200: 1, 300: 2, 500: 3, 550: 4,
    600: 5, 700: 6, 800: 7, 7100: 8, 10000: 9
}

# --- DATASET LOADER ---
class TestDataset(Dataset):
    def __init__(self, root_dir):
        self.image_dir = os.path.join(root_dir, 'Color_Images')
        # Handle case sensitivity
        p1 = os.path.join(root_dir, 'segmentation')
        p2 = os.path.join(root_dir, 'Segmentation')
        self.masks_dir = p2 if os.path.exists(p2) else p1

        self.images = sorted(glob.glob(os.path.join(self.image_dir, "*.*")))
        self.transform = A.Compose([
            A.Resize(TEST_CONFIG['IMG_SIZE'], TEST_CONFIG['IMG_SIZE']),
            A.Normalize(),
            ToTensorV2()
        ])

        # Verify
        if len(self.images) == 0:
            print(f"❌ CRITICAL ERROR: No images found in {self.image_dir}")

    def __len__(self): return len(self.images)

    def map_mask(self, mask):
        new_mask = np.zeros_like(mask, dtype=np.uint8)
        for k, v in ID_TO_INDEX.items(): new_mask[mask == k] = v
        return new_mask

    def __getitem__(self, idx):
        img_path = self.images[idx]
        filename = os.path.basename(img_path)
        mask_path = os.path.join(self.masks_dir, os.path.splitext(filename)[0] + ".png")

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Try loading mask
        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, -1)
            mask = self.map_mask(mask)
        else:
            # If no mask (blind test), return zeros
            mask = np.zeros(image.shape[:2], dtype=np.uint8)

        aug = self.transform(image=image, mask=mask)
        return aug['image'], aug['mask'].long(), image, filename

# --- METRIC CALCULATION ---
def compute_iou(pred, target, num_classes=10):
    pred = torch.argmax(pred, dim=1).view(-1)
    target = target.view(-1)
    iou_list = []
    # Ignore background class 0 if needed, but usually included
    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float()
        union = (pred_inds | target_inds).sum().float()

        if union == 0:
            iou_list.append(float('nan')) # Class not present in this image
        else:
            iou_list.append((intersection / union).item())
    return np.nanmean(iou_list), iou_list

# --- MAIN EXECUTION ---
def run_test_evaluation():
    print(f"🚀 Loading Model from: {TEST_CONFIG['MODEL_PATH']}")
    if not os.path.exists(TEST_CONFIG['MODEL_PATH']):
        print("❌ Error: Model file missing.")
        return

    # Load Model
    model = smp.UnetPlusPlus(
        encoder_name=TEST_CONFIG['ENCODER'], in_channels=3, classes=TEST_CONFIG['NUM_CLASSES']
    ).to(TEST_CONFIG['DEVICE'])
    model.load_state_dict(torch.load(TEST_CONFIG['MODEL_PATH'], map_location=TEST_CONFIG['DEVICE']))
    model.eval()

    # Load Data
    ds = TestDataset(TEST_CONFIG['TEST_DIR'])
    loader = DataLoader(ds, batch_size=1, shuffle=False)
    print(f"📂 Found {len(ds)} Test Images in 'test_public_80'")

    os.makedirs(TEST_CONFIG['OUTPUT_DIR'], exist_ok=True)
    viz_dir = os.path.join(TEST_CONFIG['OUTPUT_DIR'], "visualizations")
    os.makedirs(viz_dir, exist_ok=True)

    inference_times = []
    all_ious = []
    class_ious = {i: [] for i in range(TEST_CONFIG['NUM_CLASSES'])}

    print("⚡ Running Inference on Test Set...")
    with torch.no_grad():
        for i, (img_t, mask_t, original_img, fname) in enumerate(tqdm(loader)):
            img_t, mask_t = img_t.to(TEST_CONFIG['DEVICE']), mask_t.to(TEST_CONFIG['DEVICE'])

            # 1. Inference Speed
            start = time.time()
            logits = model(img_t)
            end = time.time()
            inference_times.append((end - start) * 1000)

            # 2. Metrics
            # Note: We only calculate IoU if the mask is not all zeros (valid ground truth)
            if mask_t.max() > 0 or mask_t.sum() > 0:
                mean_iou, cls_iou_list = compute_iou(logits, mask_t, TEST_CONFIG['NUM_CLASSES'])
                all_ious.append(mean_iou)
                for cls_idx, val in enumerate(cls_iou_list):
                    if not np.isnan(val): class_ious[cls_idx].append(val)

            # 3. Visualizations (Save first 10 + any with specific filename if you want)
            if i < 10:
                pred_mask = torch.argmax(logits, dim=1).squeeze().cpu().numpy().astype(np.uint8)

                # Create Color Mask
                h, w = pred_mask.shape
                viz_pred = np.zeros((h, w, 3), dtype=np.uint8)
                for k, v in COLOR_MAP.items(): viz_pred[pred_mask == k] = v

                # Resize original to 512 for stacking
                orig_rez = cv2.resize(original_img[0].numpy(), (512, 512))

                # Stack: Original | Prediction
                viz_final = np.hstack([orig_rez, viz_pred])

                save_path = os.path.join(viz_dir, f"pred_{fname[0]}")
                cv2.imwrite(save_path, cv2.cvtColor(viz_final, cv2.COLOR_RGB2BGR))

    # --- REPORT GENERATION ---
    avg_speed = np.mean(inference_times)
    final_mIoU = np.nanmean(all_ious) if all_ious else 0.0

    print("\n" + "="*40)
    print(f"🏆 TEST SET RESULTS (test_public_80)")
    print("="*40)
    print(f"✅ Mean IoU (mIoU):  {final_mIoU:.4f}")
    print(f"⚡ Avg Inference:    {avg_speed:.2f} ms")

    # Per-Class Table
    res_data = []
    for i in range(TEST_CONFIG['NUM_CLASSES']):
        score = np.mean(class_ious[i]) if class_ious[i] else 0.0
        res_data.append({"Class": CLASS_NAMES[i], "IoU": score})

    df = pd.DataFrame(res_data)
    print("\n📊 Per-Class Breakdown:")
    print(df.to_string(index=False))

    # Save Report
    with open(f"{TEST_CONFIG['OUTPUT_DIR']}/final_test_report.txt", "w") as f:
        f.write(f"Test Set Evaluation\n")
        f.write(f"Mean IoU: {final_mIoU:.4f}\n")
        f.write(f"Speed: {avg_speed:.2f} ms\n\n")
        f.write(df.to_string())

    print(f"\n📁 Results saved to: {TEST_CONFIG['OUTPUT_DIR']}")

# Run It
run_test_evaluation()

🚀 Loading Model from: c:/Users/Rayan/Downloads/Compressed/unzipped_files-20260214T145034Z-1-001/unzipped_files/submission1/best_model_unetpp.pth
📂 Found 738 Test Images in 'test_public_80'
⚡ Running Inference on Test Set...


 76%|███████▋  | 564/738 [15:35<04:48,  1.66s/it]


KeyboardInterrupt: 